In [9]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# pn.widgets.Tabulator?

In [51]:
import panel as pn
import pandas as pd
from datetime import datetime

pn.extension('tabulator')


class QueueTracker:

    def __init__(self, process_queue):
        self.process_queue = process_queue
        self.processed_queue = pd.DataFrame(columns=["model_id", 'function', "features", 'added_to_queue', 'start_time',  "completed_time"])

        # Tabulator tables
        self.process_table = pn.widgets.Tabulator(self.process_queue,layout ='fit_data')
        self.processed_table = pn.widgets.Tabulator(self.processed_queue, layout ='fit_data')

        # Start the periodic callback
        self.callback_id = pn.state.add_periodic_callback(self.update_tables, period=1000)  # Update every second

    def update_tables(self):
        # Update process_table
        self.process_table.value = self.process_queue.copy()
        
        # Update processed_table
        self.processed_table.value = self.processed_queue.copy()

    def mark_processed(self, record, startTime):
        """
        Mark a record as processed. It will remove the record from process_queue and 
        add it to the processed_queue with the time of completion.
        """
       
        # Add to processed_queue
        print(record)
        record['start_time'] = startTime
        record['completed_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        proccesed =  pd.DataFrame([record])    
       
        self.processed_queue = pd.concat([self.processed_queue, proccesed], ignore_index=True)
 

    def view(self):
        """Display the tables."""
        return pn.Column(
            pn.pane.Markdown("## Process Queue"),
            self.process_table,
            pn.pane.Markdown("## Processed Queue"),
            self.processed_table
        )


# Initialize
# tracker.view().servable()

NotificationArea()

In [52]:
import time
import threading
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

class QueueProcessor:

    def __init__(self, queue, tracker):
    
        self.queue = queue
        self.tracker = tracker
        self.stop_event = threading.Event()

    def process(self):
        while not self.stop_event.is_set():
            if not self.queue.empty:
                # Process the first row (or use the last row based on your preference)
                record = self.queue.iloc[0].to_dict()
                
                startTime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                self.process_record(record)
                
                # Notify QueueTracker
                self.tracker.mark_processed(record, startTime)
                
                # Remove the processed row from the queue
                self.queue.drop(self.queue.index[0], inplace=True)
            else:
                # logging.info("Queue is empty. Sleeping...")
                time.sleep(5)  # Sleep for a while if the queue is empty

    def process_record(self, record):
        # Your processing logic here
        time.sleep(2)
        logging.info("enter_processor")
        logging.info(record)
        time.sleep(5)  # For simulation, you can remove this later
        logging.info("Process Completed")
        
        
    def start(self):
        self.thread = threading.Thread(target=self.process)
        self.thread.start()

    def stop(self):
        self.stop_event.set()
        self.thread.join()
        
# processor = QueueProcessor(process_queue)
# processor.start()

In [53]:

from  formviewer import *
import data
performance_df = data.get_performace_df()
summary_df = data.get_summary_df()
model_ids, df = data.get_model_df()

global_vars = {
'current_active_tab': 0  ,
'info_max_height': 200,
'plot_height': 400,
}
# process_queue = pd.DataFrame(columns=["model_id", "features"])
process_queue = pd.DataFrame(columns=["model_id", "function", "features", "added_to_queue"])

form = FormSidebar('101', summary_df, process_queue)
tracker = QueueTracker(process_queue)
if 'processor' in globals():
    processor.stop()
processor = QueueProcessor(process_queue, tracker)
processor.start()

In [ ]:
 tracker.view()

In [64]:
form.view()

Row
    [0] Column
        [0] Select(disabled=True, name='Model ID', options=['101', '102', ...], value='110')
        [1] Button(clicks=3, name='Change', width=100)
        [2] MultiChoice(name='Add Features (CSV)', options=['var11', 'var1', ...])
        [3] MultiChoice(name='Remove Features (CSV)', options=['var9', 'var13', ...])
        [4] Button(clicks=2, name='Preview', width=100)
        [5] Row
            [0] Button(clicks=6, name='OK', width=100)
            [1] Button(clicks=1, name='Cancel', width=100)
    [1] Column
        [0] Markdown(str)
        [1] Tabulator(show_index=False, value=      110
25  ...)

In [65]:
tracker.view()

Column
    [0] Markdown(str)
    [1] Tabulator(layout='fit_data', value=  model_id       function ...)
    [2] Markdown(str)
    [3] Tabulator(layout='fit_data', sorters=[{'field': 'added_to_queue...], value=  model_id       function ...)

In [23]:
process_queue

,model_id,function,features,added_to_queue


In [6]:
test = pd.DataFrame(columns=["model_id", "features"])
record = {
 'model_id': ['id_test2'],
  'features': [['test']]
}

index = len(test)
test.loc[index] = ['id_test', 'test']  

record = {
 'model_id': ['id_test2'],
  'features': [['test']]
}
index = len(test)
test.loc[index] = ['id_test2', 'test'] 
test



def update_table():
    table.value = process_queue.copy()

table = pn.widgets.Tabulator(process_queue, width=500)

pn.state.add_periodic_callback(update_table, period=1000)
table

,model_id,features
0,id_test,test
1,id_test2,test


In [35]:
record = {
 'model_id': ['id_test'],
  'features': ['test']
}

index = len(process_queue)
process_queue.loc[index] = ['id_test', 'test', 'dfs', 'dfsd']  

INFO:root:enter_processor
INFO:root:{'model_id': 'id_test', 'function': 'test', 'features': 'dfs', 'added_to_queue': 'dfsd'}
INFO:root:Process Completed
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\enisb\anaconda3\envs\panel\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\enisb\anaconda3\envs\panel\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\enisb\AppData\Local\Temp\ipykernel_40208\2614429610.py", line 27, in process
  File "C:\Users\enisb\AppData\Local\Temp\ipykernel_40208\1021778559.py", line 39, in mark_processed
  File "C:\Users\enisb\anaconda3\envs\panel\lib\site-packages\pandas\core\frame.py", line 709, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
  File "C:\Users\enisb\anaconda3\envs\panel\lib\site-packages\pandas\core\internals\construction.py", line 481, in dict_to_mgr
    return arrays_to_mgr(arrays, colum

{'model_id': 'id_test', 'function': 'test', 'features': 'dfs', 'added_to_queue': 'dfsd'}


In [40]:
test = pd.DataFrame(columns=["model_id", "features"])
record = {
 'model_id': ['id_test2'],
  'features': [['test']]
}

index = len(test)
test.loc[index] = ['id_test', 'test']  

record = {
 'model_id': ['id_test2'],
  'features': [['test']]
}
index = len(test)
test.loc[index] = ['id_test2', [1,3,4]] 
test.iloc[1].to_dict()

{'model_id': 'id_test2', 'features': [1, 3, 4]}

In [ ]:
# record = {
# "model_id": [new_id],
# "features": [current_vars],
# }

#  = pd.concat([self.process_queue, new_model], ignore_index=True)
# new_model =  pd.DataFrame(record)                 


In [ ]:
{'model_id': 'id_test', 'features': 'test'}

In [43]:
pd.DataFrame([test.iloc[1].to_dict()] )

,model_id,features
0,id_test2,"[1, 3, 4]"
